In [2]:
#day22
#
#https://adventofcode.com/2021/day/22


def getRange(x):
    v = x.split("..")
    return int(v[0]), int(v[1])

def getAxis(x):
    v = x.split("=")
    return getRange(v[1])

def parse(text):
    #print(text)
    inst=[]
    lines=text.split("\n")
    for l in lines:
        state, rng= l.split(" ")
        #print(state, rng)
        axes=rng.split(",")
        #print(axes)
        vals = [getAxis(a) for a in axes]
        v = []
        for a,b in vals:
            v.append(a)
            v.append(b)
        v= tuple(v)
        inst.append((state,v))
    return inst

#Make sure the volume makes sense
def checkvol(vol):
    x0,x1,y0,y1,z0,z1 = vol
    assert(x0<=x1 and y0<=y1 and z0<=z1)

def touches(a, b):
    checkvol(a)
    checkvol(b)
    ax0,ax1,ay0,ay1,az0,az1 = a
    bx0,bx1,by0,by1,bz0,bz1 = b
    #a is left of b
    if ax1 < bx0 or ay1 < by0 or az1 < bz0:
        return False
    #a is right of b
    if bx1 < ax0 or by1 < ay0 or bz1 < az0:
        return False
    return True

    
#true if b fits inside a
def contains(a, b):
    checkvol(a)
    checkvol(b)
    ax0,ax1,ay0,ay1,az0,az1 = a
    bx0,bx1,by0,by1,bz0,bz1 = b
    a_low  = ax0 <= bx0 and ay0 <= by0 and az0 <= bz0
    a_high = ax1 >= bx1 and ay1 >= by1 and az1 >= bz1
    return a_low and a_high

#the common volume between two volumes (order does not matter)
def intersection(a, b):     
    ax0,ax1,ay0,ay1,az0,az1 = a
    bx0,bx1,by0,by1,bz0,bz1 = b    
    if touches(a, b):
        return (max(ax0,bx0), min(ax1, bx1), max(ay0, by0), min(ay1, by1), max(az0, bz0), min(az1, bz1))            
    return None

def cube2vol(cube):
    cx,cy,cz,cr = cube    
    return (cx, cx+cr-1, cy, cy+cr-1, cz, cz+cr-1)

def voxels(vol):
    checkvol(vol)
    x0,x1,y0,y1,z0,z1 = vol
    return (x1-x0)*(y1-y0)*(z1-z0)

class Volume:
    counter = 0
    def __init__(s, cube, depth=0):        
        s.cube = cube
        s.id = Volume.counter
        x,y,z,r = cube
        #r must must be binary round
        assert(r in [2**x for x in range(128)])        
        s.v = r**3
        s.depth=depth
        s.children = [None, None, None, None, None, None, None, None]
        s.childcubes = []
        s.genchildcubes()
        #print(s.id)
        Volume.counter += 1
   
    def __del__(s):
        pass
        #print("del ", s.id)
        
    def print(s, msg=None):
        if not msg is None:
            print(msg)
        print("id:%2d depth:%2d v:%8d v:%8d c:%d cube:%s "%(s.id, s.depth, s.v, s.count(), s.childcount(), "".join(["%4d"%(x) for x in s.cube])))
        for c in s.children:
            if not c is None:
                c.print(msg)
    
    #Generate ranges
    def genchildcubes(s):
        x,y,z,r = s.cube
        if r > 1:
            h = int(r/2)
            for xx in [x, x+h]:
                for yy in [y, y+h]:
                    for zz in [z, z+h]:
                        s.childcubes.append((xx, yy, zz, h)) 
    
    def childcount(s):
        return sum(map(lambda x : not x is None, s.children))
    
    #turn on cells
    def on(s,vol):
        checkvol(vol)
        sv = cube2vol(s.cube)
        if sv == vol:
            s.children = [None for c in s.children]
        else:
            for i in range(len(s.childcubes)):
                childcube = s.childcubes[i]
                childvol = cube2vol(childcube)                
                if touches(childvol,vol):
                    if s.children[i] is None:
                        s.children[i] = Volume(childcube, s.depth+1)
                    s.children[i].on(intersection(childvol,vol))

    #turn off cells
    def off(s,vol):
        checkvol(vol)
        sv = cube2vol(s.cube)
        
        #We are a leaf node, leaf nodes means fully on.
        #If we have to deal with deletion, we need to spawn
        #nodes to deal with subdivision
        if s.childcount() == 0:
            for i in range(len(s.childcubes)):
                childcube = s.childcubes[i]
                childvol = cube2vol(childcube)
                s.children[i] = Volume(childcube, s.depth+1)
        
        #delete nodes 
        for i in range(len(s.childcubes)):
            childcube = s.childcubes[i]
            childvol = cube2vol(childcube)                

            if touches(childvol,vol):
                #deletion fits child, so we let it go
                if contains(vol, childvol):
                    s.children[i] = None
                else:
                    if not s.children[i] is None:
                        s.children[i].off(intersection(childvol,vol))
    
    def nodes(s):
        acc = 1
        for c in s.children:
            if not c is None:
                acc += c.nodes()
        return acc
    
    #return number of cells on 
    def count(s, vol=None, debug=False):
        if vol is None:
            vol = cube2vol(s.cube)
        checkvol(vol)
        if debug:
            s.print("count")        
        if s.childcount() == 0:
            if s.depth == 0:
                return 0
            else:
                return s.v        
        acc = 0
        for c in s.children:
            if not c is None:
                v = cube2vol(c.cube)
                if touches(v, vol):
                    v = intersection(v,vol)
                    acc += c.count(v, debug)
        return acc
    
def test(tree, v, goal, text):
    if v != goal:
        print("%30s Test failed!  (returned %d when expected %d )"%(text, v, goal))
        tree.print()
        assert(v == goal)
    else:
        print("%30s Test OK"%(text))
        
            
def runtests():         
    O = -64
    R = 128
    
    print("Mini test")
    root = Volume((O,O,O, R))    
    
    root.on((2,3,2,3,2,3))
    test(root, root.count(), 8, "Turn on 8 pixles voxel")
    
    root.off((3,3,2,3,2,3))
    test(root, root.count(), 4, "Turn of 4 pixles voxel")
    
    
    
    print("Slightly larger test")
    root = Volume((O,O,O, R))    
    
    root.on((5,5,5,5,5,5))
    test(root, root.count(), 1, "Turn on single voxel")
    
    root.on((5,5,3,3,5,5))
    test(root, root.count(), 2, "Turn on another single voxel")
    
    root.on((5,5,7,7,5,5))
    test(root, root.count(), 3, "Turn on another single voxel")
    
    root.on(cube2vol((3, 3, 3, 3)))    
    test(root, root.count(), 28, "overwrite two first and second")
    
    root.on((9,9,9,9,9,9))
    test(root, root.count(), 29, "add another voxel")
    
    root.off((4,5,4,5,4,5))
    test(root, root.count(), 21, "turn off 8 voxels")
    
    root.off((3,3,4,4,3,3))
    test(root, root.count(), 20, "turn off single crooked voxel")
    
    print("Tests passed!")
    
runtests()
    
def solve1(text, presumed):
    O = -128
    R = 256
    root = Volume((O,O,O, R))    
    root.print()
    
    boot_cube = (-50,-50,-50, 101)
    boot_vol = cube2vol(boot_cube)
    print("boot_cube:", boot_cube)
    print("boot_vol:", boot_vol)
    
    instructions = parse(text)
    il = len(instructions)
    lastcount = 0
    for i in range(il):
        inst = instructions[i]
        print(inst)
        o, v = inst
        #print("operation has %d voxels"%(voxels(v)))
        if o == "on":
            root.on(v)
        else:
            root.off(v)
        count = root.count()
        nodes = root.nodes()
        eff = 0
        if nodes > 0:
            eff = int((count*1.0)/nodes)
        print("%0.0f%% tree now has %d nodes (voxels/node: %d) and %d on voxels (delta:%d)"%(i*100/(il-1.0), nodes, eff, count, count-lastcount ))
        lastcount = count
        
        #print(root.count())
    r=50
    result = root.count((-r,r,-r,r,-r,r))
    print("Bootarea count: %d"%(result))
    if not presumed is None:
        if presumed != result:
            print("result should have been: ", presumed)
            assert(0)

solve1(open("i22_test.txt").read(), 590784)    
solve1(open("i22.txt").read(), None)    
    
    

Mini test
        Turn on 8 pixles voxel Test OK
        Turn of 4 pixles voxel Test OK
Slightly larger test
          Turn on single voxel Test OK
  Turn on another single voxel Test OK
  Turn on another single voxel Test OK
overwrite two first and second Test OK
             add another voxel Test OK
             turn off 8 voxels Test OK
 turn off single crooked voxel Test OK
Tests passed!
id:62 depth: 0 v:16777216 v:       0 c:0 cube:-128-128-128 256 
boot_cube: (-50, -50, -50, 101)
boot_vol: (-50, 50, -50, 50, -50, 50)
('on', (-20, 26, -36, 17, -47, 7))
0% tree now has 10049 nodes (voxels/node: 13) and 139590 on voxels (delta:139590)
('on', (-20, 33, -21, 23, -26, 28))
5% tree now has 16517 nodes (voxels/node: 11) and 188250 on voxels (delta:48660)
('on', (-22, 28, -29, 23, -38, 16))
10% tree now has 24449 nodes (voxels/node: 7) and 195044 on voxels (delta:6794)
('on', (-46, 7, -6, 46, -50, -1))
14% tree now has 29833 nodes (voxels/node: 9) and 282552 on voxels (delta:87508)
('on'

AssertionError: 